### Librerías:

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import copy

Para los datos Landsat 8 utilizamos las siguientes fórmulas de MNDWI y NDVI:  

*NDWI = (Verde – SWIR)/(Verde + SWIR)*   
**NDWI = (Banda 3 – Banda 5)/(Banda 3 + Banda 5)** 
Los valores del NDWI corresponden a los siguientes rangos:  
0,2 – 1 – Superficie del agua,  
0,0 – 0,2 – Inundación, humedad,  
-0,3 – 0,0 – Sequía moderada, superficies sin agua,  
-1 – -0,3 – Sequía, superficies sin agua  

*NDVI = (Rojo – NIR)/(Rojo + NIR)*
**NDVI = (Banda 4 – Banda 5)/(Banda 4 + Banda 5)**  


Carga de imagenes

In [6]:
# Cargamos la imagen y la dividimos por 255 para normalizarla 
path = 'Los-Molinos_2023-05-14_6_all_bands/'
B3= mpimg.imread(path + '2023-05-14.SR_B2.tif') / 255
B4= mpimg.imread(path + '2023-05-14.SR_B2.tif') / 255
B3= mpimg.imread(path + '2023-05-14.SR_B2.tif') / 255